In [ ]:
import mip

In [4]:
from mip import Model, xsum, CONTINUOUS, MINIMIZE

# Example data
workers = ["Alice", "Bob", "Charlie"]
tasks = ["Task1", "Task2", "Task3"]

# Cost matrix: cost[i][j] = cost of worker i doing task j (per unit time)
cost = [
    [8, 6, 10],   # Alice
    [7, 5, 9],    # Bob
    [6, 8, 7]     # Charlie
]

# Task requirements: fraction of total worker time needed for each task
task_demand = [1.0, 0.5, 1.5]  # e.g., Task1 needs 1 full-time equivalent, Task2 needs 0.5, etc.

# Worker capacity: max fraction of time each worker can work (1.0 = full-time)
worker_capacity = [1.0, 1.0, 1.0]

# Create model
m = Model(sense=MINIMIZE)

# Decision variables: fraction of worker i's time assigned to task j
x = [[m.add_var(var_type=CONTINUOUS, lb=0, ub=1, name=f"x_{i}_{j}")
      for j in range(len(tasks))] for i in range(len(workers))]

# Objective: minimize total cost
m.objective = xsum(cost[i][j] * x[i][j] for i in range(len(workers)) for j in range(len(tasks)))

# Constraint: each task's demand must be met
for j in range(len(tasks)):
    m += xsum(x[i][j] for i in range(len(workers))) == task_demand[j]

# Constraint: each worker's total assigned time ≤ capacity
for i in range(len(workers)):
    m += xsum(x[i][j] for j in range(len(tasks))) <= worker_capacity[i]

# Solve
m.optimize()

# Output results
if m.status.value == 0:
    print(f"Total cost: {m.objective_value:.2f}")
    for i in range(len(workers)):
        for j in range(len(tasks)):
            if x[i][j].x > 1e-6:  # avoid printing near-zero values
                print(f"{workers[i]} works {x[i][j].x:.2f} on {tasks[j]}")
else:
    print("No optimal solution found.")


Starting solution of the Linear programming problem using Primal Simplex

Coin0506I Presolve 6 (0) rows, 9 (0) columns and 18 (0) elements
Clp1000I sum of infeasibilities 2.07142e-05 - average 3.45236e-06, 1 fixed columns
Coin0506I Presolve 5 (-1) rows, 7 (-2) columns and 14 (-4) elements
Clp0006I 0  Obj 21.999954 Primal inf 1.6714156e-05 (4) Dual inf 3e+12 (7)
Clp0029I End of values pass after 7 iterations
Clp0000I Optimal - objective value 22
Clp0000I Optimal - objective value 22
Coin0511I After Postsolve, objective 22, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 22
Clp0000I Optimal - objective value 22
Clp0000I Optimal - objective value 22
Clp0000I Optimal - objective value 22
Clp0032I Optimal objective 22 - 0 iterations time 0.002, Idiot 0.00
Total cost: 22.00
Alice works 0.50 on Task2
Alice works 0.50 on Task3
Bob works 1.00 on Task1
Charlie works 1.00 on Task3


In [5]:
from mip import Model, xsum, CONTINUOUS, MINIMIZE

# Example data
workers = ["Alice", "Bob", "Charlie"]
tasks = ["Task1", "Task2", "Task3"]

# Fractional availability (max hours each worker can work)
availability = {"Alice": 8, "Bob": 6, "Charlie": 5}

# Task hours required
task_hours = {"Task1": 5, "Task2": 4, "Task3": 6}

# Cost per hour for each worker-task pair
cost = {
    ("Alice", "Task1"): 10, ("Alice", "Task2"): 12, ("Alice", "Task3"): 15,
    ("Bob", "Task1"): 9,  ("Bob", "Task2"): 14, ("Bob", "Task3"): 20,
    ("Charlie", "Task1"): 11, ("Charlie", "Task2"): 13, ("Charlie", "Task3"): 18
}

# Credentials: 1 if worker is qualified for task, else 0
qualified = {
    ("Alice", "Task1"): 1, ("Alice", "Task2"): 1, ("Alice", "Task3"): 0,
    ("Bob", "Task1"): 1, ("Bob", "Task2"): 0, ("Bob", "Task3"): 1,
    ("Charlie", "Task1"): 0, ("Charlie", "Task2"): 1, ("Charlie", "Task3"): 1
}

# Model
m = Model(sense=MINIMIZE)

# Decision variables: hours assigned (continuous)
x = {(w, t): m.add_var(var_type=CONTINUOUS, lb=0,
                       ub=availability[w] if qualified[(w, t)] else 0)
     for w in workers for t in tasks}

# Objective: minimize total cost
m.objective = xsum(cost[(w, t)] * x[(w, t)] for w in workers for t in tasks)

# Task requirements: sum of hours from all workers = required hours
for t in tasks:
    m += xsum(x[(w, t)] for w in workers) == task_hours[t]

# Worker availability: total hours assigned ≤ availability
for w in workers:
    m += xsum(x[(w, t)] for t in tasks) <= availability[w]

# Solve
m.optimize()

# Output
for w in workers:
    for t in tasks:
        if x[(w, t)].x > 1e-6:
            print(f"{w} works {x[(w, t)].x:.2f}h on {t}")

print(f"Total cost: {m.objective_value:.2f}")


Starting solution of the Linear programming problem using Primal Simplex

Coin0506I Presolve 2 (-4) rows, 2 (-7) columns and 4 (-14) elements
Clp1000I sum of infeasibilities 0 - average 0, 2 fixed columns
Coin0506I Presolve 0 (-2) rows, 0 (-2) columns and 0 (-4) elements
Clp0000I Optimal - objective value 203
Clp0000I Optimal - objective value 203
Coin0511I After Postsolve, objective 203, infeasibilities - dual 0 (0), primal 0 (0)
Clp0006I 0  Obj 203
Clp0000I Optimal - objective value 203
Clp0000I Optimal - objective value 203
Clp0000I Optimal - objective value 203
Coin0511I After Postsolve, objective 203, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 203 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00
Alice works 4.00h on Task2
Bob works 5.00h on Task1
Bob works 1.00h on Task3
Charlie works 5.00h on Task3
Total cost: 203.00


python mip assign worker fractional time to task, each worker has different credentials, each task requires different credential working hours


model an assignment problem with fractional work allocation in Python using MIP, where:

Workers can split their time across tasks (fractional assignment).
Each worker has certain credentials (skills).
Each task requires a certain number of hours for each credential type.

In [6]:
from mip import Model, xsum, CONTINUOUS, minimize

# Example data
workers = ["Alice", "Bob", "Charlie"]
tasks = ["Task1", "Task2"]

# Worker credentials (hours they can work per credential type)
worker_capacity = {
    "Alice": {"A": 6, "B": 4},
    "Bob": {"A": 5, "B": 5},
    "Charlie": {"A": 4, "B": 6}
}

# Task requirements (hours needed per credential type)
task_requirements = {
    "Task1": {"A": 5, "B": 3},
    "Task2": {"A": 4, "B": 5}
}

# Cost per hour for each worker-task assignment
cost = {
    ("Alice", "Task1"): 10,
    ("Alice", "Task2"): 12,
    ("Bob", "Task1"): 9,
    ("Bob", "Task2"): 11,
    ("Charlie", "Task1"): 8,
    ("Charlie", "Task2"): 10
}

# Create model
m = Model(sense=minimize)

# Decision variables: hours worker w spends on task t
x = {(w, t): m.add_var(var_type=CONTINUOUS, lb=0) for w in workers for t in tasks}

# Credential constraints: each task's credential requirement must be met
for t in tasks:
    for cred in task_requirements[t]:
        m += xsum(x[w, t] for w in workers if cred in worker_capacity[w]) >= task_requirements[t][cred]

# Worker capacity constraints: total hours per credential type
for w in workers:
    for cred in worker_capacity[w]:
        m += xsum(x[w, t] for t in tasks if cred in task_requirements[t]) <= worker_capacity[w][cred]

# Objective: minimize total cost
m.objective = xsum(cost[w, t] * x[w, t] for w in workers for t in tasks)

# Solve
m.optimize()

# Output results
for w in workers:
    for t in tasks:
        if x[w, t].x > 1e-6:
            print(f"{w} works {x[w, t].x:.2f} hours on {t}")

print(f"Total cost: {m.objective_value:.2f}")


Starting solution of the Linear programming problem using Primal Simplex

Coin0506I Presolve 5 (-5) rows, 6 (0) columns and 12 (-12) elements
Clp1000I sum of infeasibilities 0 - average 0, 0 fixed columns
Coin0506I Presolve 5 (0) rows, 6 (0) columns and 12 (0) elements
Clp0006I 0  Obj 97 Dual inf 6000 (6)
Clp0029I End of values pass after 6 iterations
Clp0000I Optimal - objective value 97
Clp0000I Optimal - objective value 97
Clp0000I Optimal - objective value 97
Coin0511I After Postsolve, objective 97, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 97 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00
Alice works 1.00 hours on Task2
Bob works 1.00 hours on Task1
Bob works 4.00 hours on Task2
Charlie works 4.00 hours on Task1
Total cost: 97.00


model an assignment problem in Python using MIP where:

Workers can split their time fractionally across tasks.
Each worker has certain credentials (skills).
Each task requires a certain number of hours for each credential.
The model should also detect worker shortages if total available hours for a credential are less than required.
Here’s an example formulation using python-mip that captures this:

Python-MIP fractional assignment with credential constraints & shortage detection

In [8]:
from mip import Model, xsum, CONTINUOUS, minimize

# Example data
workers = ["Alice", "Bob", "Charlie"]
tasks = ["Task1", "Task2"]

# Worker available hours
avail_hours = {"Alice": 8, "Bob": 6, "Charlie": 5}

# Worker credentials
worker_creds = {
    "Alice": {"A": 1, "B": 0},
    "Bob": {"A": 1, "B": 0},
    "Charlie": {"A": 1, "B": 0}
}

# Task credential requirements (hours)
task_reqs = {
    "Task1": {"A": 6, "B": 4},
    "Task2": {"A": 4, "B": 3}
}

m = Model()

# Decision variables: fraction of worker's time assigned to a task
assign = {(w, t): m.add_var(var_type=CONTINUOUS, lb=0) for w in workers for t in tasks}

# Shortage variables for each credential
shortage = {c: m.add_var(var_type=CONTINUOUS, lb=0) for c in {"A", "B"}}

# Worker time limits
for w in workers:
    m += xsum(assign[w, t] for t in tasks) <= avail_hours[w]

# Credential coverage constraints
for t in tasks:
    for c in {"A", "B"}:
        m += xsum(assign[w, t] * worker_creds[w][c] for w in workers) + shortage[c] >= task_reqs[t][c]

# Objective: minimize total shortage
m.objective = minimize(xsum(shortage[c] for c in shortage))

m.optimize()

# Output results
print("\nAssignments:")
for w in workers:
    for t in tasks:
        if assign[w, t].x > 1e-6:
            print(f"{w} -> {t}: {assign[w, t].x:.2f} hours")

print("\nShortages:")
for c in shortage:
    print(f"Credential {c}: {shortage[c].x:.2f} hours")


Starting solution of the Linear programming problem using Dual Simplex

Clp0024I Matrix will be packed to eliminate 6 small elements
Coin0506I Presolve 2 (-5) rows, 2 (-6) columns and 4 (-12) elements
Clp0006I 0  Obj 4 Primal inf 5.999999 (1)
Clp0000I Optimal - objective value 4
Coin0511I After Postsolve, objective 4, infeasibilities - dual 0 (0), primal 0 (0)

Assignments:
Alice -> Task2: 8.00 hours
Bob -> Task1: 6.00 hours
Charlie -> Task2: 5.00 hours

Shortages:
Credential B: 4.00 hours
Credential A: 0.00 hours
Clp0032I Optimal objective 4 - 1 iterations time 0.002, Presolve 0.00
